# 빠른 시작 가이드 - Azure AI Foundry

이 노트북은 Azure AI Foundry에 대한 실습 소개를 제공합니다:
1. AI 프로젝트 클라이언트 초기화
2. 사용 가능한 모델 나열
3. 간단한 채팅 완료 요청을 생성
4. 기본 AI 에이전트 생성
5. 기본 오류 시나리오 다루기

## 전제 조건
- 이전 notebook에서 환경 설정 완료
- Azure 자격 증명 구성

## 필수 라이브러리 및 설정 가져오기

다음 코드의 내용은 다음을 수행합니다:
1. 인증 및 AI 프로젝트에 필요한 Azure SDK 라이브러리를 가져옵니다
2. 환경 변수 및 JSON 처리를 위한 표준 Python 라이브러리를 가져옵니다
3. DefaultAzureCredential을 사용하여 Azure 자격 증명을 초기화
   - 로그인한 Azure CLI 자격 증명을 자동으로 사용
   - 또는 환경 변수 또는 매니지드 아이덴티티와 같은 다른 인증 방법을 사용할 수 있습니다.


In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials
credential = DefaultAzureCredential()

## AI Project Client 초기화

> **참고:** 계속 진행하기 전에 다음 사항을 확인하세요:
> 1. 루트 디렉토리의 `.env.example` 파일을 `.env` 로 복사합니다.
> 2. `.env` 파일에서 프로젝트 연결 문자열을 업데이트합니다.
> 3. Azure AI Foundry에 허브 및 프로젝트를 프로비전하여 배포해서 가지고 있어야 합니다.

프로젝트의 설정 아래에서 [Azure AI Foundry](https://ai.azure.com)프로젝트 연결 문자열을 찾을 수 있습니다:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## AI Project Client 생성하기

다음 코드에서는 `.env` 파일의 연결 문자열을 사용하여 AI 프로젝트 클라이언트를 만들겠습니다.
> **주의:** 이 예제에서는 동기식 클라이언트를 사용합니다. 더 높은 성능의 시나리오를 위해 `asyncio`를 임포트하고 `AIProjectClient`의 비동기 메서드를 사용하여 비동기 클라이언트를 생성할 수도 있습니다.

클라이언트는 다음에 사용합니다:
- 연결 문자열을 사용하여 Azure AI 프로젝트에 연결
- Azure 자격 증명을 사용하여 인증
- 배포된 모델에 추론 요청을 사용하도록 설정


In [ ]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

try:
    client = AIProjectClient.from_connection_string(
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

## 간단한 완료 요청 만들기
기본적인 완료 요청을 해보겠습니다:

이제 인증된 클라이언트에서 채팅 완료 요청을 만들어 보겠습니다. 
아래 코드는 그 방법을 보여줍니다:
1. azure-ai-inference 패키지에서 ChatCompletionsClient를 가져옵니다.
2. 이를 사용하여 간단한 완료 요청을 만듭니다.

코드를 변경하지 않고도 배포된 다른 모델 간에 쉽게 전환할 수 있도록 `.env` 파일의
MODEL_DEPLOYMENT_NAME을 사용하겠습니다. Azure OpenAI 모델, Microsoft 모델
또는 채팅 완료를 지원하는 기타 제공업체가 될 수 있습니다.

> 참고: azure-ai-inference 패키지가 설치되어 있는지 확인하세요(requirements.txt 또는 [README.md](../README.md#-quick-start)에 언급된 대로)


In [ ]:
from azure.ai.inference.models import UserMessage

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

try:
    chat_client = client.inference.get_chat_completions_client()
    response = chat_client.complete(
        model=model_deployment_name, 
        messages=[UserMessage(content="How to be healthy in one sentence?")]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {str(e)}")

## 간단한 에이전트 만들기

AI 에이전트 서비스를 사용하여 건강 관련 질문에 답하는 간단한 에이전트를 만들 수 있습니다.

인텔리전트 에이전트를 빌드하기 위한 강력한 도구인 Azure AI 에이전트 서비스에 대해 알아봅시다.

Azure AI 에이전트 서비스는 개발자가 인프라를 관리하지 않고도 AI 에이전트를 빌드, 배포 및 확장할 수 있도록 지원하는 완전 관리형 서비스입니다.
이 서비스는 대규모 언어 모델과 에이전트가 다음을 수행할 수 있는 도구를 결합합니다:
- RAG(Retrieval Augmented Generation)를 사용하여 질문에 답하기
- 툴 호출을 통해 작업 수행
- 복잡한 워크플로우 자동화

아래 코드는 그 방법을 보여줍니다:
1. 코드 인터프리터 툴로 에이전 만들기
2. 대화 스레드 만들기
3. BMI 분석을 요청하는 메시지 보내기
4. 요청을 처리하고 결과 얻기
5. 생성된 이미지 결과 저장하기

에이전트는 `.env` 파일(MODEL_DEPLOYMENT_NAME)에 지정된 모델을 사용하며 시각화를
만들기 위한 코드 인터프리터 툴에 액세스할 수 있습니다.
이는 에이전트가 자연어 이해와 계산 기능을 결합하는 방법을 보여줍니다.

> 이미지 결과는 이 노트북과 같은 폴더에 PNG 파일로 저장됩니다.
 



In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

try:
    # Initialize the Code Interpreter Tool
    code_interpreter = CodeInterpreterTool()
    
    # Create an AI agent with the code interpreter tool
    agent = client.agents.create_agent(
        model=model_deployment_name,
        name="bmi-calculator",
        instructions=(
            "You are a health analyst who calculates BMI using US metrics (pounds, feet/inches). "
            "Use average US female measurements: 5'4\" (69 inches) and 130 pounds. "
            "Create a visualization showing where this BMI falls on the scale."
        ),
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    
    # Create a new conversation thread
    thread = client.agents.create_thread()
    
    # Create a message requesting BMI analysis and visualization
    message = client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(
            "Calculate BMI for an average US female (5'4\", 130 lbs). "
            "Create a visualization showing where this BMI falls on the standard BMI scale from 15 to 35. "
            "Include the standard BMI categories (Underweight, Normal, Overweight, Obese) in the visualization."
        )
    )
    
    # Process the request by creating and running a thread run
    run = client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    
    # Retrieve and save any generated visualizations
    messages = client.agents.list_messages(thread_id=thread.id)
    for image_content in messages.image_contents:
        file_name = f"bmi_analysis_{image_content.image_file.file_id}.png"
        client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Analysis saved as: {file_name}")
    
    # Print the analysis text from the assistant
    print(f"Messages: {messages}")
    if last_msg := messages.get_last_text_message_by_role("assistant"):
        print(f"Analysis: {last_msg.text.value}")
    
    # Cleanup by deleting the agent
    client.agents.delete_agent(agent.id)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")